In [1]:
# Check if All required packages are installed, 
# If not, install them
import sys
import subprocess
import pkg_resources
# The List of required packages
# If an import requires a package to be installed, it Must be in this list, or the End User will have issues
required_packages = {'tk', 'pillow', 'pathlib2', 'rasterio', 'openai','pyperclip', 'google-api-python-client', 'google-cloud', \
                     'google-cloud-aiplatform', 'google-cloud-vision'}# Get the list of Installed Packages
installed_packages = {package.key for package in pkg_resources.working_set}
# Get a list of all the Required Packages that are not Installed
missing_packages = required_packages - installed_packages

# If Missing packages is not Empty
if missing_packages:
    # Print the list of Missing Packages
    print ("Missing: ",missing_packages)
    # Create an System Excecutable object that allows calling 'pip install <package>'
    python_installer = sys.executable
    # Itterate through Missing packages and Install them
    # Note: Some Imports come from Packages that do not share the same name.
    #   - Those Packages are noted below above the Import that they belong to
    for pkg in missing_packages:
        try:
            print("Installing: ", pkg)
            # Install the Current Package
            subprocess.check_call([python_installer, '-m', 'pip', 'install', pkg, '-q'], stdout=subprocess.DEVNULL)
            # Print the Installed Package, on a Failure, an Error will occor.
            print("Installed: ", pkg)
        except subprocess.CalledProcessError:
            print("Could not Install: ", pkg)
else:
    print("All Packages are Installed")

All Packages are Installed


In [2]:
# install tk
from tkinter import *
from tkinter import ttk, StringVar, font
# install pillow
from PIL import Image, ImageTk
import os
from os import getcwd
from os.path import isfile, join, abspath
import io
from pathlib import Path
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import openai
import urllib
import pprint
#import cv2
import json
import pyperclip
from google.cloud import aiplatform,vision
from vertexai.preview.language_models import CodeGenerationModel
import requests


In [3]:
# Get the path to the CWD. This will be used as a Master Refference for everything else
directoryPath = abspath(getcwd())
# OpenAI model details
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://financial-poc-capegmini.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_KEY"] = "147fa51fedb346babf30e8e91b0ad8e5"

openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]

# GCP API
starchat_URL = 'https://w58yxognl8.execute-api.us-east-1.amazonaws.com/V2/starchat-beta'



In [4]:
prompt_modelA_textbox = None
prompt_modelB_textbox = None
prompt_textbox = None
expected2_textbox = None
expected3_textbox = None
time2_md_textBox = None
time3_md_textBox = None
o_response1_textbox = None
o_response2_textbox = None
s_response1_textbox = None
s_response2_textbox = None
root = None
choice_var = None
save_response_button = None
modelA_API = "Default A\n"
modelB_API = "Default B\n"
title_font =('Segoe UI',12)
save_default_str = "Save Review"
#{'family': 'Segoe UI', 'size': 9, 'weight': 'normal', 'slant': 'roman', 'underline': 0, 'overstrike': 0}


In [5]:
def refresh_root():
    root.update()

In [6]:
def copy_model_A():
    pyperclip.copy(prompt_modelA_textbox.get("1.0","end-1c"))

In [7]:
def copy_model_B():
    pyperclip.copy(prompt_modelB_textbox.get("1.0","end-1c"))

In [8]:
def write_model_A():
    output_file = open("model_a_test.py","w")
    output_file.write(prompt_modelA_textbox.get("1.0","end-1c"))
    output_file.close

In [9]:
def write_model_B():
    output_file = open("model_b_test.py","w")
    output_file.write(prompt_modelB_textbox.get("1.0","end-1c"))
    output_file.close

In [10]:
def call_model_OpenAI(code):
    model_API = "# OpenAI\n"
    print("Model {code}:")
    print("API KEY: " , os.environ["OPENAI_API_KEY"])

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt_textbox.get(1.0, "end-1c"),
        temperature=.1,
        max_tokens=1000,
        top_p=0.1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        n=1,
        stop=None)
    if code == "A":
        prompt_modelA_textbox.delete("1.0","end")
        prompt_modelA_textbox.insert("end",model_API + response.choices[0].text)
    elif code == "B":
        prompt_modelB_textbox.delete("1.0","end")
        prompt_modelB_textbox.insert("end",model_API + response.choices[0].text)
    #print(response.choices[0].text)
    

In [11]:
def generate_a_function(temperature=0.5,prefix="Write a function that checks if a year is a leap year.") -> object:
    """Example of using Code Generation to write a function."""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
    }

    code_generation_model = CodeGenerationModel.from_pretrained("code-bison@001")
    response = code_generation_model.predict(
        prefix=prefix, **parameters
    )

    print(f"Response from Model: {response.text}")
    # [END aiplatform_sdk_code_generation_function]

    return response

In [12]:
def call_model_StarChat(code):
    
    model_API = "# StarChat\n"
    print("Model {code}:")
    starchat_body = {
        "prompt":prompt_textbox.get(1.0, "end-1c")
        }
    starchat_response = requests.post(starchat_URL, json = starchat_body)
    starchat_json = starchat_response.json()
    print(starchat_json['body'])
    if code == "A":
        prompt_modelA_textbox.delete("1.0","end")
        prompt_modelA_textbox.insert("end",model_API + starchat_json['body'])
    elif code == "B":
        prompt_modelB_textbox.delete("1.0","end")
        prompt_modelB_textbox.insert("end",model_API + starchat_json['body'])

In [13]:
def call_models():
    """change which lines are commented out to change which models are called in each box"""
    call_model_OpenAI("A")
    #call_model_OpenAI("B")
    #call_model_StarChat("A")
    call_model_StarChat("B")
    #call_model_GCP("A") ...WIP...
    #call_model_GCP("B") ...WIP...
    root.update()

In [14]:
def write_output():
    output_file = open("output.txt","w")
    choice_str = ""

    if choice_var == 1:
        choice_str = "Model A is better\n"
    elif choice_var == 2:
        choice_str = "Model B is better\n"
    elif choice_var == 3:
        choice_str = "It's a tie\n"
    elif choice_var == 4:
        choice_str = "Neither are good\n"
    else:
        choice_str = "Defalt Choice\n"

    write_str = "Prompt:\n" + prompt_textbox.get("1.0","end-1c") + "\n\n" + \
                    "Model A:\n" + prompt_modelA_textbox.get("1.0","end-1c") + "\n\n" + \
                    "Model B:\n" + prompt_modelB_textbox.get("1.0","end-1c") + "\n\n" + \
                    "Review:\n" + \
                    "" + choice_str + \
                    "% Closer to the expected output:\n" +\
                    "Model A: " + expected2_textbox.get("1.0","end-1c") + "%\n" + \
                    "Model B: " + expected3_textbox.get("1.0","end-1c") + "%\n" + \
                    "Aditional time needed:\n"+\
                    "Model A: " + time2_md_textBox.get("1.0","end-1c") + " OR ...WIP...\n" + \
                    "Model B: " + time3_md_textBox.get("1.0","end-1c") + " OR ...WIP...\n" + \
                    "Optimization criteria:\n"+\
                    "Model A: " + o_response1_textbox.get("1.0","end-1c") + "\n" + \
                    "Model B: " + o_response2_textbox.get("1.0","end-1c") + "\n" + \
                    "Extra comments:\n"+\
                    "Model A: " + s_response1_textbox.get("1.0","end-1c") + "\n" + \
                    "Model B: " + s_response2_textbox.get("1.0","end-1c") + "\n" + \
                    "..."

                
    output_file.write(write_str)
    output_file.close()

In [15]:
def save_click():
    print("Choice:",choice_var.get())
    if choice_var.get() == 0:
        print("Make a selection")
        save_response_button.config(bg='thistle2',state='disabled')
    else:
        write_output()
        save_response_button.config(bg='light blue',text="Saved",state='disabled')
    root.update()


In [16]:
def check_click():
    print("Choice:",choice_var.get())
    if choice_var.get() == 0:
        print("Make a selection")
        save_response_button.config(bg='thistle2',text=save_default_str,state='disabled')
    else:
        save_response_button.config(bg='light green',text=save_default_str,state='active')
    root.update()

In [17]:
root: Tk = Tk()
root.title("GenAI Comparison Testing")
screen_width: int = root.winfo_screenwidth()
screen_height: int = root.winfo_screenheight()
root.geometry("%dx%d" % (screen_width, screen_height))

root.configure(background='#fff')

#background_image_src = Image.open('bg_grad_purple_blue.png')
#background_image = ImageTk.PhotoImage(background_image_src)
button_bg_image_src = Image.open('button_bg1.png')
button_bg_image = ImageTk.PhotoImage(button_bg_image_src)

#background_image_label = Label(root,image=background_image)
#background_image_label.place(x=0,y=0,width=screen_width,height=screen_height)

#String Variables
hours_arr = range(0,23)
minutes_arr = range(0,59)
hours_str_arr = []
minutes_str_arr = []
for i in hours_arr: 
    hours_str_arr.append(str(i))
for i in minutes_arr: 
    minutes_str_arr.append(str(i))
time2_hrs_var: StringVar = StringVar(root)
time2_hrs_var.set('0')
time2_min_var: StringVar = StringVar(root)
time2_min_var.set('0')
time3_hrs_var: StringVar = StringVar(root)
time3_hrs_var.set('0')
time3_min_var: StringVar = StringVar(root)
time3_min_var.set('0')
prompt_var: StringVar = StringVar(root)
default_prompt_message = "generate python code to merge two dataframes"
prompt_var.set(default_prompt_message)

# Intager Variables
choice_var = IntVar()
choice_var.set(0)

#Prompt Master Frame
#prompt_master_frame = Frame(root,highlightbackground="orange", highlightthickness=2,background='#fff')
prompt_master_frame = Frame(root,background='#fff')
prompt_master_frame.grid(column=0,row=0,sticky=N,padx=2)

#Prompt Text Frame
prompt_text_frame = Frame(prompt_master_frame,background='#fff')
prompt_text_frame.grid(column=0,row=0)
prompt_title_label = Label(prompt_text_frame, text="Prompt the models",font=title_font,bg='#fff')
prompt_title_label.grid(column=0,row=0)
prompt_sub_label = Label(prompt_text_frame,justify=LEFT,text="For the given task above, write a prompt in the prompt box below \n" +\
                         "to generate outputs from both of the Models A and B.",bg='#fff')
prompt_sub_label.grid(column=0,row=1)

#Prompt Entry Labels
text_width = 25
text_height = 20
prompt_width = int((text_height * 2.5) + 2)
prompt_height = text_height / 4
prompt_entry_frame = Frame(prompt_master_frame,background='#fff')
prompt_entry_frame.grid(column=0,row=1,padx=5,pady=5)
prompt_modelA_label = Label(prompt_entry_frame,text="Model A",bg='#fff')
prompt_modelA_label.grid(column=0,row=0)
prompt_modelB_label = Label(prompt_entry_frame,text="Model B",bg='#fff')
prompt_modelB_label.grid(column=1,row=0)
prompt_modelA_textbox = Text(prompt_entry_frame,width=text_width,height=text_height,highlightbackground='#888', highlightthickness=1)
prompt_modelA_textbox.grid(column=0,row=1,padx=5,pady=5)
prompt_modelB_textbox = Text(prompt_entry_frame,width=text_width,height=text_height,highlightbackground='#888', highlightthickness=1)
prompt_modelB_textbox.grid(column=1,row=1,padx=5,pady=5)
prompt_modelA_copybutton = Button(prompt_entry_frame,text="Write A",command=write_model_A,background='LightSkyBlue1',borderwidth=0)
prompt_modelA_copybutton.grid(column=0,row=2,padx=5,pady=5)
prompt_modelB_copybutton = Button(prompt_entry_frame,text="Write B",command=write_model_B,background='LightSkyBlue1',borderwidth=0)
prompt_modelB_copybutton.grid(column=1,row=2,padx=5,pady=5)
prompt_textbox = Text(prompt_entry_frame,width=prompt_width,height=prompt_height,highlightbackground='#888', highlightthickness=1)
prompt_textbox.grid(column=0,row=3,pady=15,columnspan=2)
prompt_textbox.delete("1.0","end")
prompt_textbox.insert("end",default_prompt_message)
# Submit Prompt Button
submit_button_frame = Frame(prompt_entry_frame,background='#fff')
submit_button_frame.grid(column=0,row=4,columnspan=2,sticky=E,padx=5,pady=2)
refresh_button = Button(submit_button_frame,text="Refresh",command=refresh_root,width=8,height=1,background='#ddf',borderwidth=0)
refresh_button.grid(column=0,row=0,sticky=E)
submit_button = Button(submit_button_frame,text="Submit",command=call_models,width=8,height=1,background='light green',borderwidth=0)
submit_button.grid(column=1,row=0,sticky=E)


root_separator = ttk.Separator(root,orient=VERTICAL)
root_separator.grid(column=1,row=0,sticky=NS)

# Question Frame
question_master_frame = Frame(root,background='#fff')
question_master_frame.grid(column=2,row=0,padx=5,sticky=N)

question_header_frame = Frame(question_master_frame,background='#fff')
question_header_frame.grid(column=0,row=0,padx=5)

question_header_label = Label(question_header_frame,text="Answer the following questions",bg='#fff',font=title_font)
question_header_label.grid(column=0,row=0)
question_sub_label = Label(question_header_frame,text="Which models' output buality is better?*",bg='#fff')
question_sub_label.grid(column=0,row=1)

# Choice Frame
question_choice_frame = Frame(question_master_frame,background='#fff')
question_choice_frame.grid(column=0,row=2)

option1_rb = Radiobutton(question_choice_frame, text="Model A is better",variable=choice_var,value=1,command=lambda: [SEL,check_click()],bg='#fff')
option1_rb.grid(column=0,row=0)
option2_rb = Radiobutton(question_choice_frame, text="Model B is better",variable=choice_var,value=2,command=lambda: [SEL,check_click()],bg='#fff')
option2_rb.grid(column=1,row=0)
option3_rb = Radiobutton(question_choice_frame, text="It's a tie",variable=choice_var,value=3,command=lambda: [SEL,check_click()],bg='#fff')
option3_rb.grid(column=2,row=0)
option4_rb = Radiobutton(question_choice_frame, text="Neither are good",variable=choice_var,value=4,command=lambda: [SEL,check_click()],bg='#fff')
option4_rb.grid(column=3,row=0)

# Question Detail Frame
question_detail_frame = Frame(question_master_frame,highlightbackground="#888", highlightthickness=1,background='#fff')
question_detail_frame.grid(column=0,row=3,padx=2,sticky=EW)

detail_header_padx = 10
detail_header_pady = 5
detail_header1_label = Label(question_detail_frame, text="       ",bg='#fff',font=title_font)
detail_header1_label.grid(column=0,row=0,padx=detail_header_padx,pady=detail_header_pady)
vertical1_seperator = ttk.Separator(question_detail_frame,orient=VERTICAL)
vertical1_seperator.grid(column=1,row=0,rowspan=5,sticky=NS)
detail_header2_label = Label(question_detail_frame, text="Model A",bg='#fff',font=title_font)
detail_header2_label.grid(column=2,row=0,padx=detail_header_padx,pady=detail_header_pady)
vertical2_seperator = ttk.Separator(question_detail_frame,orient=VERTICAL)
vertical2_seperator.grid(column=3,row=0,rowspan=5,sticky=NS)
detail_header3_label = Label(question_detail_frame, text="Model B",bg='#fff',font=title_font)
detail_header3_label.grid(column=4,row=0,padx=detail_header_padx,pady=detail_header_pady)
detail_header_seperator = ttk.Separator(question_detail_frame,orient=HORIZONTAL)
detail_header_seperator.grid(column=0,row=1,columnspan=5,sticky=EW)

expected1_frame = Frame(question_detail_frame,background='#fff')
expected1_frame.grid(column=0,row=2,padx=2,sticky=NW)
expected1_header_label = Label(expected1_frame,anchor='w',text="% Closer to the expected output",bg='#fff')
expected1_header_label.grid(column=0,row=0,padx=2,sticky=W)
expected1_sub_label = Label(expected1_frame,justify=LEFT,text="Both models have generated outputs based on your prompts.\n" + \
                        "How close are the outputs to the expected output?\n" + \
                        "Enter in %.",bg='#fff')
expected1_sub_label.grid(column=0,row=1,padx=2,sticky=W)

percentage_width = 12

expected2_frame = Frame(question_detail_frame,background='#fff')
expected2_frame.grid(column=2,row=2,padx=2,sticky=N)
expected2_header_label = Label(expected2_frame,text="Percentage*",bg='#fff')
expected2_header_label.grid(column=0,row=0,padx=2)
expected2_textbox = Text(expected2_frame,width=percentage_width,height=1,highlightbackground='#888', highlightthickness=1)
expected2_textbox.grid(column=0,row=1,padx=2)

expected3_frame = Frame(question_detail_frame,background='#fff')
expected3_frame.grid(column=4,row=2,padx=2,sticky=N)
expected3_header_label = Label(expected3_frame,text="Percentage*",bg='#fff')
expected3_header_label.grid(column=0,row=0,padx=2)
expected3_textbox = Text(expected3_frame,width=percentage_width,height=1,highlightbackground='#888', highlightthickness=1)
expected3_textbox.grid(column=0,row=1,padx=2)
expected_seperator = ttk.Separator(question_detail_frame,orient=HORIZONTAL)
expected_seperator.grid(column=0,row=3,columnspan=5,sticky=EW)

time1_frame = Frame(question_detail_frame,background='#fff')
time1_frame.grid(column=0,row=4,padx=2,sticky=NW)
time1_header_label = Label(time1_frame,justify=LEFT,text="Time needed to be spent on improving the generated output",bg='#fff')
time1_header_label.grid(column=0,row=0,padx=2,sticky=W)
time1_sub_label = Label(time1_frame,justify=LEFT,text="How much time would you\n" + \
                        "have to spend to modify the model generated\n" + \
                        "outputs to match your expected output?",bg='#fff')
time1_sub_label.grid(column=0,row=1,padx=2,sticky=W)

time2_frame = Frame(question_detail_frame,background='#fff')
time2_frame.grid(column=2,row=4,padx=2,sticky=N)
time2_md_label = Label(time2_frame,text="Man Days*",bg='#fff')
time2_md_label.grid(column=0,row=0,padx=1)
time2_md_textBox = Text(time2_frame,width=5,height=1,highlightbackground='#888', highlightthickness=1)
time2_md_textBox.grid(column=0,row=1,padx=2)
time2_or_label = Label(time2_frame,text="OR ",bg='#fff')
time2_or_label.grid(column=1,row=1)
time2_hrs_label = Label(time2_frame,text="Hrs*",bg='#fff')
time2_hrs_label.grid(column=2,row=0,padx=2)
time2_hrs_opt = ttk.Combobox(time2_frame, textvariable=time2_hrs_var, values=hours_str_arr,width=5)
time2_hrs_opt.grid(column=2,row=1,padx=2)
time2_min_label = Label(time2_frame,text="Min*",bg='#fff')
time2_min_label.grid(column=3,row=0,padx=2)
time2_min_opt = ttk.Combobox(time2_frame, textvariable=time2_min_var, values=minutes_str_arr,width=5)
time2_min_opt.grid(column=3,row=1,padx=2)

time3_frame = Frame(question_detail_frame,background='#fff')
time3_frame.grid(column=4,row=4,padx=2,sticky=N)
time3_md_label = Label(time3_frame,text="Man Days*",bg='#fff')
time3_md_label.grid(column=0,row=0,padx=1)
time3_md_textBox = Text(time3_frame,width=5,height=1,highlightbackground='#888', highlightthickness=1)
time3_md_textBox.grid(column=0,row=1,padx=2)
time3_or_label = Label(time3_frame,text="OR ",bg='#fff')
time3_or_label.grid(column=1,row=1)
time3_hrs_label = Label(time3_frame,text="Hrs*",bg='#fff')
time3_hrs_label.grid(column=2,row=0,padx=2)
time3_hrs_opt = ttk.Combobox(time3_frame, textvariable=time3_hrs_var, values=hours_str_arr,width=5)
time3_hrs_opt.grid(column=2,row=1,padx=2)
time3_min_label = Label(time3_frame,text="Min*",bg='#fff')
time3_min_label.grid(column=3,row=0,padx=2)
time3_min_opt = ttk.Combobox(time3_frame, textvariable=time3_min_var, values=minutes_str_arr,width=5)
time3_min_opt.grid(column=3,row=1,padx=2)

# Optomize Frame
optomize_master_frame = Frame(question_master_frame,background='#fff')
optomize_master_frame.grid(column=0,row=4,padx=2)
optomize_header_label = Label(optomize_master_frame,font=title_font,text="What criteria do you need to optimize the outputs for?*",bg='#fff')
optomize_header_label.grid(column=0,row=0,pady=2)
optomize_sub_label = Label(optomize_master_frame,text="For each model, can you list the areas where the outputs need to\n" +\
                        "be optimized further? List at least one point each.",bg='#fff')
optomize_sub_label.grid(column=0,row=1,pady=2)
optomize_response_frame = Frame(optomize_master_frame,background='#fff')
optomize_response_frame.grid(column=0,row=2,padx=5,pady=5)
response_width = 30
response_height = 3
o_response1_label = Label(optomize_response_frame,text="Model A*",bg='#fff')
o_response1_label.grid(column=0,row=0,padx=5,pady=3)
o_response1_textbox = Text(optomize_response_frame,width=response_width,height=response_height,highlightbackground='#888', highlightthickness=1)
o_response1_textbox.grid(column=0,row=1,padx=2,pady=2)
o_response2_label = Label(optomize_response_frame,text="Model B*",bg='#fff')
o_response2_label.grid(column=1,row=0,padx=5,pady=3)
o_response2_textbox = Text(optomize_response_frame,width=response_width,height=response_height,highlightbackground='#888', highlightthickness=1)
o_response2_textbox.grid(column=1,row=1,padx=2,pady=2)

# Standards Frame
standards_master_frame = Frame(question_master_frame,background='#fff')
standards_master_frame.grid(column=0,row=5,padx=2)
standards_header_label = Label(standards_master_frame,font=title_font,text="Extra comments?",bg='#fff')
standards_header_label.grid(column=0,row=0,pady=2)
#standards_sub_label = Label(standards_master_frame,text="Can you list for each model the areas on which the outputs need to\n" +\
#                        "be optimized further for it to be as you envision. List at least one point each.r",bg='#fff')
#standards_sub_label.grid(column=0,row=1,pady=2)
standards_response_frame = Frame(standards_master_frame,background='#fff')
standards_response_frame.grid(column=0,row=1,padx=5,pady=5)
response_width = 30
response_height = 3
s_response1_label = Label(standards_response_frame,text="Model A",bg='#fff')
s_response1_label.grid(column=0,row=0,padx=5,pady=3)
s_response1_textbox = Text(standards_response_frame,width=response_width,height=response_height,highlightbackground='#888', highlightthickness=1)
s_response1_textbox.grid(column=0,row=1,padx=2,pady=2)
s_response2_label = Label(standards_response_frame,text="Model B",bg='#fff')
s_response2_label.grid(column=1,row=0,padx=5,pady=3)
s_response2_textbox = Text(standards_response_frame,width=response_width,height=response_height,highlightbackground='#888', highlightthickness=1)
s_response2_textbox.grid(column=1,row=1,padx=2,pady=2)

save_response_button = Button(question_master_frame,text=save_default_str,command=save_click,background='thistle2',borderwidth=0,state='disabled')
save_response_button.grid(column=0,row=6,padx=2,ipadx=5)

root.lift()
#root.attributes('-topmost', True)
root.mainloop()


Model {code}:
API KEY:  147fa51fedb346babf30e8e91b0ad8e5
Model {code}:

Here is an example of Python code that merges two dataframes based on a common column:

import pandas as pd

# Load the two dataframes
df1 = pd.read_csv('dataframe1.csv')
df2 = pd.read_csv('dataframe2.csv')

# Merge the dataframes based on a common column
merged_df = pd.merge(df1, df2, on='common_column')

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_dataframe.csv', index=False)

In this example, the dataframes df1 and df2 are loaded using the Pandas read_csv function, and then merged using the Pandas merge function. The on parameter specifies the common column on which to merge the dataframes. The merged dataframe is then saved to a CSV file using the to_csv function.
